In [4]:
!pip install tensorflow==2.15.0

  Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl.metadata (3.6 kB)
  Using cached tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl (300.9 MB)

   ---------------------------------------- 0/2 [tensorflow-intel]
   ---------------------------------------- 0/2 [tensorflow-intel]
   ---------------------------------------- 0/2 [tensorflow-intel]



ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\meowm\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\quantization\\tensorflow\\python\\pywrap_quantize_model.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import tensorflow as tf
def remove_unwanted_groups(model):
    def custom_clone(layer):
        if isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            config = layer.get_config()
            config.pop('groups', None)  # Remove 'groups' if it exists
            new_layer = tf.keras.layers.DepthwiseConv2D.from_config(config)
            new_layer.build(layer.input_shape)
            new_layer.set_weights(layer.get_weights())
            return new_layer
        return layer.__class__.from_config(layer.get_config())

    return tf.keras.models.clone_model(model, clone_function=custom_clone)
float_model = tf.keras.models.load_model("models/YOLO_x_nano/st_yolo_x_nano_192_0.33_0.25.h5", compile=False)
float_model = remove_unwanted_groups(float_model)
float_model.save("testtest.h5")

TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'stack1_dw_conv', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [2, 2], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}

In [1]:
from neural_compressor.tensorflow import quantize_model, StaticQuantConfig
from neural_compressor.data import DataLoader
import os
import tensorflow as tf
import numpy as np
import math
import random
from pathlib import Path
from glob import glob

def _get_image_paths(dataset_root: str = None, shuffle: bool = True, seed: int = None) -> list:
    """
    Gets all the paths to .jpg image files under a dataset root directory.

    If the function is called with the `shuffle` argument set to True
    and without the `seed` argument, or with the `seed` argument set 
    to None, the file paths are shuffled but results are not reproducible.
    
    if the `shuffle` argument is set to False, paths are sorted
    in alphabetical order.

    Arguments:
        dataset_root:
            A string, the path to the directory that contains the image files.
        shuffle:
            A boolean, specifies whether file paths should be shuffled or not.
            Defaults to True.
        seed:
            An integer, the seed to use to make paths shuffling reproducible.
            Used only when `shuffle` is set to True.
    
    Returns:
        A list of image file paths.
    """

    if not os.path.isdir(dataset_root):
        raise ValueError(f"Unable to find dataset directory {dataset_root}")
        
    jpg_file_paths = glob(os.path.join(Path(dataset_root), "*.jpg"))
    if not jpg_file_paths:
        raise ValueError(f"Could not find any .jpg image files in directory {dataset_root}")

    if shuffle:
        random.seed(seed)
        random.shuffle(jpg_file_paths)
    else:
        jpg_file_paths.sort()

    return jpg_file_paths

def _create_image_loader(
            image_paths: list,
            image_size: tuple = None,
            batch_size: int = None,
            rescaling: tuple = None,
            interpolation: str = None,
            aspect_ratio: str = None,
            color_mode: str = None) -> tf.data.Dataset:
            
    """
    Creates a tf.data.Dataset data loader for images.
    
    Arguments:
        image_paths:
            List of paths to image files.
        image_size:
            A tuple of 2 integers: (width, height).
            Specifies the size of the images supplied by
            the data loader.
        batch_size:
            An integer, the size of data batches supplied
            by the data loader.
        rescaling:
            A tuple of 2 floats: (scale, offset). Specifies
            the factors to use to rescale the input images.
        interpolation:
            A string, the interpolation method to use to resize
            the input images.
        aspect_ratio:
            A string, the aspect ratio method to use to resize
            the input images (fit, crop, pad).
        color_mode:
            A string, the color mode (rgb or grayscale).

    Returns:
        A tf.data.Dataset data loader.
    """
    
    def _load_image(img_path):
    
        # Load the input image
        channels = 1 if color_mode == "grayscale" else 3
        data = tf.io.read_file(img_path)
        image_in = tf.io.decode_jpeg(data, channels=channels)
        if color_mode.lower() == 'bgr':
            image_in = image_in[...,::-1]
        
        # Resize the input image
        width_out = image_size[0]
        height_out = image_size[1]
        if aspect_ratio == "fit":
            image_out = tf.image.resize(image_in, (height_out, width_out), method=interpolation)
        else:
            image_out = tf.image.resize_with_crop_or_pad(image_in, height_out, width_out)
        
        # Rescale the output image
        image_out = tf.cast(image_out, tf.float32)
        # image_out = rescaling[0] * image_out + rescaling[1]
        if isinstance(rescaling[0], (list, tuple)) or isinstance(rescaling[1], (list, tuple)):
            # Assuming rescaling[0] and rescaling[1] are lists/tuples of length equal to number of channels
            variances = tf.constant(rescaling[0], dtype=tf.float32)
            means = tf.constant(rescaling[1], dtype=tf.float32)
            image_out = (image_out / variances ) + means
        else:
            # Default scalar rescaling
            image_out = rescaling[0] * image_out + rescaling[1]
        
        return image_out, tf.convert_to_tensor(img_path)
        
    ds = tf.data.Dataset.from_tensor_slices(image_paths)
    ds = ds.map(_load_image)
    #ds = ds.batch(batch_size)
    return ds

def get_quantization_data_loader(
                image_size: tuple = None,
                batch_size: int = 1,
                image_paths_only: bool = False,
                seed: int = None,
                verbose: bool = True) -> tf.data.Dataset:
    """
    Creates a data loader for quantizing a float model.

    The dataset is chosen in the following precedence order:
      1. quantization set
      2. test set
      2. training set

    If a quantization split ratio was set, the chosen dataset
    is split accordingly. Otherwise, it is used entirely.
    If no dataset is available, the function returns None.
    In this case, quantization will be done using fake data.

    Arguments:
        cfg:
            A dictionary, the entire configuration file dictionary.
        image_size:
            A tuple of 2 integers: (width, height).
            Specifies the size of the images supplied by
            the data loaders.
        batch_size:
            An integer, the size of data batches supplied by the data
            loader. Defaults to 1.
        seed:
            An integer, the seed to use to make file paths shuffling and
            dataset splitting reproducible.
            Defaults to cfg.dataset.seed
        verbose:
            A boolean. If True, the dataset path and size are displayed.
            If False, no message is displayed.
            Default to True.

    Returns:
        A tf.data.Dataset data loader
    """

    # Quantize with fake data if quantization_split is set to 0
    quantization_split = 0.02
    quantization_path = val
    if quantization_split is not None and quantization_split == 0:
        return None
     
    # Look for a dataset
    if quantization_path:
        ds_path = quantization_path
    elif cds.training_path:
        ds_path = cds.training_path
    else:
        # No dataset available, quantize with fake data
        return None 
    
    if not seed:
        seed = 123       
    image_paths = _get_image_paths(ds_path, shuffle=True, seed=seed)
    
    if cds.quantization_split:
        num_images = int(len(image_paths) * quantization_split)
        percent_used = "{:.3f}%".format(100 * quantization_split)
    else:
        # quantization_split is not set, get the entire dataset.
        num_images = len(image_paths)
        percent_used = "100% (use quantization_split to choose a different percentage)"
        
    image_paths = image_paths[:num_images]

    if verbose:
        print("Quantization dataset:")
        print("  path:", ds_path)
        print(f"  percentage used: {percent_used}")
        print(f"  number of images: {num_images}")

    if not image_paths_only:
        cpp = cfg.preprocessing
        quantization_ds = _create_image_loader(
                        image_paths,
                        image_size=image_size,
                        batch_size=batch_size,
                        rescaling=(1/255, 0),
                        interpolation="nearest",
                        aspect_ratio="fit",
                        color_mode="greyscale")
        return quantization_ds
    else:
        return image_paths

def _quantize_neurComp_model():
    """
        Quantize a keras model using Intel Neural Compressor

        Args:
            cfg (DictConfig): Entire configuration file.
            model_path: Optional[str]: Path to the float model

        Returns:
            quantized model path
        """
    print("here 1")
    float_model = tf.keras.models.load_model("testtest.h5", compile=False)
    float_model = remove_unwanted_groups(float_model)
    print("here 2")
    quant_ds = get_quantization_data_loader(cfg,
                image_size=input_shape, batch_size=1)
    print("here 3")
    calib_loader = DataLoader(
        framework='tensorflow',
        dataset=quant_ds,
        batch_size=1 
    )
    print("here 4")
    out_dir = Path("Quantized_Models")
    out_dir.mkdir(parents=True, exist_ok=True)
    print("here 5")
    quant_cfg = StaticQuantConfig(
        weight_dtype='int4',
        act_dtype='int8',
        weight_granularity='per_channel',
        act_granularity='per_tensor',
        weight_sym=True,
        act_sym=True,
    )
    print("here 6")
    inp_op = float_model.input_names[0].split(':')[0]
    quant_cfg.set_local(inp_op, StaticQuantConfig(act_dtype='int8'))
    
    for tensor_name in float_model.output_names:
        op_name = tensor_name.split(':')[0]
        quant_cfg.set_local(op_name, StaticQuantConfig(act_dtype='float32'))
    print("here 7")
    qmodel = quantize_model(
        model=float_model,
        quant_config=quant_cfg,
        calib_dataloader=calib_loader
    )
    print("here 8")
    save_path = out_dir / 'quant_model'
    qmodel.save(str(save_path))
    print(str(save_path))
    return str(save_path)

quantized_model_path = _quantize_neurComp_model()
print(f"Quantized model saved at: {quantized_model_path}")

here 1


TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'stack1_dw_conv', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [2, 2], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}